In [2]:
import sys
import os
import numpy as np

# path to folder to CityGeoTools
folder = "/var/essdata/IDU/other/Rest_/refactored/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel
from metrics.calculations import CityMetricsMethods as CityMetrics

In [3]:
city_model = BaseModel.CityInformationModel(city_name="Saint-Petersburg", city_crs=32636, cwd="../")
city_model

### Create CityInformationModel

The object of class CityInformationModel contains attributes describing data required for successful method call.  
After creating own model these attributes equal to **None** by default.

In [4]:
city_model.get_all_attributes()

{'city_name': 'Saint-Petersburg',
 'city_crs': 32636,
 'city_id': None,
 'cwd': '../',
 'mode': 'user_mode',
 'MobilityGraph': None,
 'Buildings': None,
 'Services': None,
 'PublicTransportStops': None,
 'ServiceTypes': None,
 'Blocks': None,
 'Municipalities': None,
 'AdministrativeUnits': None,
 'methods': <metrics.data.DataValidation.DataValidation at 0x7fca98debac0>}

### Update layers

To use CityGeoTools methods, it is necessary to load prepaid data to the model specifying attribute names and paths to files.  
Only **json**, **geojson** and **graphml** file formats are available for loading. Before data loading, all methods are marked as unavailable.  
Exception is weighted voronoi method, since it doesn't require any urban data.

In [5]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")


All methods implemented in CityGeoTools:

traffic_calculator --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
accessibility_isochrones --> unavailable
diversity --> unavailable
collocation_matrix --> unavailable


In [8]:
# GeoJSON
city_model.update_layer("Buildings", "./data/buildings.geojson")
city_model.update_layer("Services", "./data/services.geojson")
city_model.update_layer("Blocks", "./data/blocks.geojson")

#JSON
city_model.update_layer("ServiceTypes", "./data/service_types.json")

#GraphML
city_model.update_layer("MobilityGraph", "./data/graph.graphml")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Services layer...
Services layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!
Validation of ServiceTypes layer...
ServiceTypes layer loaded successfully!
Validation of MobilityGraph layer...
MobilityGraph layer loaded successfully!


In [9]:
city_model.Services

,geometry,building_id,id,city_service_type,city_service_type_id,service_code,service_name,block_id,administrative_unit_id,municipality_id,x,y
0,POINT (350078.267 6651195.638),NaN,493918,Остановка общественного транспорта,113,stops,улица Профессора Попова,1183.0,59.0,43.0,350078.267130,6.651196e+06
1,POINT (354297.100 6657911.331),NaN,494130,Остановка общественного транспорта,113,stops,"проспект Луначарского, 86",1345.0,60.0,21.0,354297.099801,6.657911e+06
2,POINT (350825.017 6635901.548),NaN,494196,Остановка общественного транспорта,113,stops,"Московское шоссе, 16",1481.0,57.0,61.0,350825.017333,6.635902e+06
3,POINT (365365.397 6624979.037),NaN,494207,Остановка общественного транспорта,113,stops,Красная улица,1461.0,56.0,6.0,365365.397207,6.624979e+06
4,POINT (326276.132 6642328.550),NaN,494233,Остановка общественного транспорта,113,stops,Мастеровой переулок,5174.0,47.0,7.0,326276.132086,6.642329e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
74350,POINT (350758.855 6649949.098),NaN,265487,Урна,33,garbage_containers,Урна,1403.0,59.0,62.0,350758.855245,6.649949e+06
74351,POINT (351119.261 6647301.996),NaN,265971,Урна,33,garbage_containers,Урна,674.0,58.0,70.0,351119.261128,6.647302e+06
74352,POINT (357946.831 6645516.457),NaN,266027,Урна,33,garbage_containers,Урна,673.0,52.0,55.0,357946.830748,6.645516e+06
74353,POINT (349941.096 6639579.684),NaN,266057,Урна,33,garbage_containers,Урна,1459.0,57.0,59.0,349941.096323,6.639580e+06


In [11]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")

All methods implemented in CityGeoTools:

traffic_calculator --> unavailable
visibility_analysis --> available
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
accessibility_isochrones --> available
diversity --> unavailable
collocation_matrix --> unavailable


### Deal with ValidationError

It should be noted that all loaded data MUST match specification for certain method to successfully call it.  
Otherwise, you will encounter the ValidationError even though the file is saved into attribute.   
To know why loaded file does not match specification, see attribute **message** that is available trought city_model.methods  

In [12]:
CityMetrics.Spacematrix(city_model).get_spacematrix_morph_types()

ValidationError: Layers Blocks do not match specification.

In [13]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': "48.0 is not of type 'integer'"}